### 2.4

假设$Y$与$X_1,X_2$之间满足线性回归关系
$$
y_1=\beta_0+\beta_1x_{i1}+\beta_2x_{i2}+\epsilon_i,i=1,2,3,\dots,15
$$
其中$\epsilon_i \sim N(0,\sigma^2)$且独立同分布

In [13]:
import numpy as np
import pandas as pd
data = pd.DataFrame({
    '销量': [162, 120, 223, 131, 67, 169, 81, 192, 116, 55, 252, 232, 144, 103, 212],
    '人数': [274, 180, 375, 205, 86, 265, 98, 330, 195, 53, 430, 372, 236, 157, 370],
    '收入': [2450, 3254, 3802, 2838, 2347, 3782, 3008, 2450, 2137, 2560, 4020, 4427, 2660, 2088, 2605]}
)


#### (1)求回归系数$\beta_0,\beta_1,\beta_2$的最小二乘估计和误差方差$\sigma^2$的估计，写出回归方程并对回归系数作解释

已知
$$
\hat{\beta} = (\hat{\beta}_0, \hat{\beta}_1, \dots, \hat{\beta}_{p-1})^T=(X^TX)^{-1}X^TY \\
\hat{\sigma^2}=\cfrac{SSE}{n-p}=\frac{1}{n-p}Y^T(I-H)Y \\
H=X(X^TX)^{-1}X^T
$$

In [14]:
X = data[['人数', '收入']].values
# X加上一个全是1的列
X_wide = np.hstack((np.ones((X.shape[0], 1)), X))
Y = data['销量'].values
H = X_wide @ np.linalg.inv(X_wide.T @ X_wide) @ X_wide.T
beta_hat = np.linalg.inv(X_wide.T @ X_wide) @ X_wide.T @ Y
I = np.eye(H.shape[0])
sigma_2_hat = Y.T @ (I - H) @ Y / (X_wide.shape[0] - X_wide.shape[1])
print(beta_hat)
print(sigma_2_hat)

[3.45261279 0.49600498 0.00919908]
4.740297129881294


$\hat{y}=3.453+0.496x_1+0.009x_2$

$\beta_0$截距项：在“人数”和“收入”都为 0 的情况下的“销量”预测值，虽然没有现实意义，但对模型是必要的

$\beta_1$：“人数”每增加 1 单位，其他条件不变时，“销量”预计增加$\beta_1$个单位

$\beta_2$：“收入”每增加 1 单位，其他条件不变时，“销量”预计增加$\beta_2$个单位

#### (2)求出方差分析表，解释对线性回归关系显著性检验结果，求复相关系数平方$R^2$的值并解释其意义

$$
\begin{array}{|c|c|c|c|c|c|}
\hline
\text{方差来源} & \text{自由度} & \text{平方和 (SS)} & \text{均方 (MS)} & \text{F 值} & \text{p 值} \\
\hline
\text{回归 (R)} & p - 1 & SSR & MSR=\cfrac{SSR}{p - 1} & F_0=\cfrac{MSR}{MSE} & p_0 \\
\hline
\text{误差 (E)} & n - p & SSE & MSE=\cfrac{SSE}{n - p} &  &  \\
\hline
\text{总和 (T)} & n - 1 & SST &  &  &  \\
\hline
\end{array}
$$

$$
SST=Y^T(I-\frac{1}{n}J)Y=\sum_{i=1}^{n}(y_i - \bar{y})^2 \\
SSR=Y^T(H-\frac{1}{n}J)Y=\sum_{i=1}^{n}(\hat{y}_i - \bar{y})^2 \\
SSE=Y^T(I-H)Y=\sum_{i=1}^{n}(y_i - \hat{y}_i)^2 \\
SST=SSE+SSR \\
MSR=\frac{SSR}{p-1} \\
MSE=\frac{SSE}{n-p} \\
R^2=\cfrac{SSR}{SST}=1-\cfrac{SSE}{SST},R=\sqrt{R^2} \\
p_0=P(F(p-1,n-p) \geq F_0)
$$

In [15]:
SST = np.sum((Y - np.mean(Y)) ** 2)
SSR = np.sum((H @ Y - np.mean(Y)) ** 2)
SSE = np.sum((Y - H @ Y) ** 2)
R_2 = SSR / SST
MSR = SSR / (X_wide.shape[1] - 1)
MSE = SSE / (X_wide.shape[0] - X_wide.shape[1])
F = MSR / MSE
# 假设检验p值
from scipy.stats import f
p_value = 1 - f.cdf(F, X_wide.shape[1] - 1, X_wide.shape[0] - X_wide.shape[1])
# 全部输出
print('SST:', SST)
print('SSR:', SSR)
print('SSE:', SSE)
print('R^2:', R_2)
print('MSR:', MSR)
print('MSE:', MSE)
print('F:', F)
print('p-value:', p_value)

SST: 53901.6
SSR: 53844.716434440685
SSE: 56.883565559127966
R^2: 0.9989446776058722
MSR: 26922.358217220342
MSE: 4.74029712992733
F: 5679.466387718415
p-value: 1.1102230246251565e-16


$p$值很小，认为线性回归关系显著.

模型的复相关系数平方（决定系数）为$R^2$=0.998，表示模型能够解释99.8%的销量变异，说明模型拟合效果较好，变量“人数”和“收入”能很好地解释销量的变动。

#### (3)分别求分别求出$\beta_1$和$\beta_2$的95%置信区间

$$
\cfrac{\hat{\beta}_k - \beta_k}{\sigma \sqrt{c_{kk}}} \sim N(0,1) \\
t_k = \cfrac{\hat{\beta}_k - \beta_k}{\hat{\sigma} \sqrt{c_{kk}}} \sim t(n-p) \\
\hat{\sigma} \sqrt{c_{kk}} = s(\hat{\sigma}_k) \\
\hat{\beta_k} \pm t_{1-\frac{\alpha}{2}}(n-p)s(\hat{\sigma}_k)
$$

In [16]:
from scipy.stats import t
cov_beta = sigma_2_hat * np.linalg.inv(X_wide.T @ X_wide)
standard_errors = np.sqrt(np.diag(cov_beta))
n, p = X_wide.shape
t_value = t.ppf(0.975, df=n - p)
for i, name in enumerate(['β₀', 'β₁（人数）', 'β₂（收入）']):
    lower = beta_hat[i] - t_value * standard_errors[i]
    upper = beta_hat[i] + t_value * standard_errors[i]
    print(f"{name} 的95%置信区间为：[{lower:.4f}, {upper:.4f}]")

β₀ 的95%置信区间为：[-1.8433, 8.7485]
β₁（人数） 的95%置信区间为：[0.4828, 0.5092]
β₂（收入） 的95%置信区间为：[0.0071, 0.0113]


#### (4)对$\alpha=0.05$，分别检验人数$X_1$和$X_2$对销量$Y$的影响是否显著，利用与回归系数有关的一般假设检验方法检验$X_1$和$X_2$的交互作用(即$X_1X_2$)对$Y$的影响是否显著

$$
Y = \beta_0 + \beta_1 X_1 + \beta_2 X_2 + \beta_3 X_1 X_2 + \varepsilon
$$

其中，$\varepsilon \sim N(0, \sigma^2)$，$X_1$ 表示“人数”，$X_2$ 表示“收入”，$X_1X_2$ 为交互项。

我们构造扩展设计矩阵 $X \in \mathbb{R}^{n \times 4}$，每行的形式为：

$$X_i = \begin{bmatrix} 1 & x_{i1} & x_{i2} & x_{i1}x_{i2} \end{bmatrix}$$

回归系数的最小二乘估计为：

$$\hat{\beta} = (X^T X)^{-1} X^T Y$$

残差为：

$$\varepsilon = Y - \hat{Y} = Y - X \hat{\beta}$$

残差平方和（SSE）为：

$$SSE = \sum_{i=1}^{n}(y_i - \hat{y}_i)^2 = \varepsilon^T \varepsilon$$

误差方差的估计值为：

$$\hat{\sigma}^2 = \frac{SSE}{n - p}$$

回归系数的协方差矩阵估计为：

$$\text{Cov}(\hat{\beta}) = \hat{\sigma}^2 (X^T X)^{-1}$$

第 $k$ 个回归系数的标准误差为：

$$SE(\hat{\beta}_k) = \sqrt{[\text{Cov}(\hat{\beta})]_{kk}}$$

对每个回归系数进行假设检验：

$$H_{0k}:\beta_k = 0 \quad \text{vs} \quad H_{1k}:\beta_k \ne 0$$

检验统计量为：

$$t_k = \frac{\hat{\beta}_k}{SE(\hat{\beta}_k)} \sim t(n - p)$$

计算双尾 $p$ 值：

$$p_{0k} = 2P(t(n - p) > |t_k|)$$

给定显著性水平 $\alpha = 0.05$，比较 $|t_k|$ 与临界值 $t_{0.975}(n - p)$：

$$\text{若 } |t_k| > t_{0.975}(n - p) \Rightarrow \text{拒绝 } H_{0k} \Rightarrow \text{变量显著}$$

我们分别对 $\beta_1$（人数）、$\beta_2$（收入）以及 $\beta_3$（交互项）进行上述检验，以判断它们对销量的显著性影响。


In [17]:
# 添加交互项：X3 = 人数 * 收入
X1 = data['人数'].values
X2 = data['收入'].values
X3 = X1 * X2
X_ext = np.column_stack((np.ones(len(X1)), X1, X2, X3))  # 添加常数项

Y = data['销量'].values.reshape(-1, 1)

# 重新计算 beta_hat
XtX_inv_ext = np.linalg.inv(X_ext.T @ X_ext)
beta_hat_ext = XtX_inv_ext @ X_ext.T @ Y

# 残差与 sigma2
Y_hat_ext = X_ext @ beta_hat_ext
residuals_ext = Y - Y_hat_ext
n_ext, p_ext = X_ext.shape
SSE_ext = (residuals_ext.T @ residuals_ext).item()
sigma2_ext = SSE_ext / (n_ext - p_ext)

# 协方差矩阵
cov_beta_ext = sigma2_ext * XtX_inv_ext
standard_errors_ext = np.sqrt(np.diag(cov_beta_ext))

# t 值与双尾 p 值
from scipy.stats import t
t_critical_ext = t.ppf(0.975, df=n_ext - p_ext)

for i, name in enumerate(['β₀', 'β₁（人数）', 'β₂（收入）', 'β₃（交互项）']):
    t_k = beta_hat_ext[i][0] / standard_errors_ext[i]
    p_val = 2 * (1 - t.cdf(np.abs(t_k), df=n_ext - p_ext))
    result = "显著" if np.abs(t_k) > t_critical_ext else "不显著"
    print(f"{name}: t = {t_k:.4f}, p = {p_val:.4f}, α = 0.05 → {result}")


β₀: t = 0.5740, p = 0.5775, α = 0.05 → 不显著
β₁（人数）: t = 17.3435, p = 0.0000, α = 0.05 → 显著
β₂（收入）: t = 2.7766, p = 0.0180, α = 0.05 → 显著
β₃（交互项）: t = 0.1777, p = 0.8622, α = 0.05 → 不显著


#### (5)该公司欲在一个适宜使用该化妆品的人数$x_{01}=200$，人均月收入$X_{02}=2500$的新的城市中销售该化妆品，求七销量的预测值及其置信度为$95%$的置信区间

设新城市中适宜使用该化妆品的人数为 $x_{01} = 200$，人均月收入为 $x_{02} = 2500$，将该点代入前述回归模型：

$$\hat{y}_0 = x_0^T \hat{\beta} = \hat{\beta}_0 + \hat{\beta}_1 \cdot 200 + \hat{\beta}_2 \cdot 2500$$


其置信度为 $95\%$ 的预测区间为：

$$\hat{y}_0 \pm t_{n-p}^{(0.975)} \cdot \sqrt{\hat{\sigma}^2 \cdot x_0^T (X^T X)^{-1} x_0}$$


In [18]:
# 新城市变量
x0 = np.array([1, 200, 2500]).reshape(1, -1)  # 加入常数项

# 预测值
y0_hat = x0 @ beta_hat.reshape(-1, 1)

# 标准误差项
x_cov = x0 @ np.linalg.inv(X_wide.T @ X_wide) @ x0.T
se_y0 = np.sqrt(sigma_2_hat * x_cov[0][0])

# t 临界值
t_critical = t.ppf(0.975, df=X_wide.shape[0] - X_wide.shape[1])

# 区间计算
lower = y0_hat[0][0] - t_critical * se_y0
upper = y0_hat[0][0] + t_critical * se_y0

print(f"预测销量为：{y0_hat[0][0]:.4f}")
print(f"95%置信区间为：[{lower:.4f}, {upper:.4f}]")


预测销量为：125.6513
95%置信区间为：[124.1876, 127.1151]
